<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

In [3]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 56.0 MB/s eta 0:00:00


**save_df_to_drive**
Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a google drive file path

In [4]:

def save_df_to_drive(df, file_path_in_drive):
    """
    Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a Google Drive file path

    parameters:
      df (pd.DataFrame): The Pandas DataFrame needing to be saved
      file_path_in_drive (str): Google Drive folder file path that will store the CSV file
    """
    try:
        # Ensure the destination directory exists
        drive.mount('/content/drive')  # Mount Google Drive to access files

        # Get the absolute path in Google Drive
        absolute_path_in_drive = os.path.join('/content/drive/My Drive', file_path_in_drive)

        # Ensure the destination directory exists in Google Drive
        os.makedirs(os.path.dirname(absolute_path_in_drive), exist_ok=True)

        # Save the DataFrame to the specified file path in Google Drive
        df.to_csv(absolute_path_in_drive, index=False)

        print(f"DataFrame saved to Google Drive at '{absolute_path_in_drive}'")

    except Exception as e:
        print("An error occurred:", str(e))

# Example usage:
# Assuming your file_path_in_drive is 'YourFolder/YourFile.csv'
# Make sure to change it according to your actual folder structure
#save_df_to_drive(your_dataframe, 'YourFolder/YourFile.csv')


**Image collection**
get_sentinel_images_ee
Function that allows the download of Sentinel-2 images from GEE

In [5]:
#import ee
#from datetime import datetime

#def get_sentinel_images_ee(latitude, longitude, start_date, end_date):
    #"""
    #Function to search and download Sentinel-2 images using Google Earth Engine.

    #Parameters:
        #latitude (float): Latitude coordinate of the desired location.
        #longitude (float): Longitude coordinate of the desired location.
        #start_date (str): Start date of the date range in the format 'YYYY-MM-DD'.
        #end_date (str): End date of the date range in the format 'YYYY-MM-DD'.

    #Returns:
        #List of image collection IDs for the retrieved Sentinel-2 images.
    #"""
    # Create a point geometry representing the location
    #point = ee.Geometry.Point(longitude, latitude)

    # Define the date range
    #date_range = ee.DateRange(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d'))

    # Filter Sentinel-2 data based on location and date range
    #collection = (ee.ImageCollection('COPERNICUS/S2')
                  #.filterBounds(point)
                  #.filterDate(date_range))

    # Get a list of image IDs in the collection
    #image_ids = collection.aggregate_array('system:id').getInfo()

    #return image_ids

# Example usage:
#latitude = -25.12  # Hazyview
#longitude = 30.88  # Hazyview
#start_date = '2023-07-27'
#end_date = '2023-07-28'

#result = get_sentinel_images_ee(latitude, longitude, start_date, end_date)
#print("Image Collection IDs:", result)

In [6]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

**Calculating indices**

In [8]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('B3'),
    'RED': image.select('B4'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('B3'),
    'GREEN': image.select('B4'),
    'BLUE': image.select('B2'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
    'BLUE': image.select('B2'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo(), 'image_bands')

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

Loading training data from GEE

In [9]:

# Load the training data (e.g., a FeatureCollection) from GEE
training_data = ee.FeatureCollection('projects/mapwaps-sabiecroc/assets/Training_setSc')

# Convert the training data to a GeoDataFrame
features = training_data.getInfo()['features']
training_subset = gpd.GeoDataFrame.from_features(features)

# Display the GeoDataFrame
print(training_subset.head())



                     geometry DIRECTION DIRECTIO_1      DateTime DateTime_1  \
0  POINT (30.83475 -25.53864)                      -2.209133e+12              
1  POINT (30.83556 -25.53828)                      -2.209133e+12              
2  POINT (30.94377 -25.06340)         E            -2.209133e+12              
3  POINT (30.93543 -25.06455)         E            -2.209133e+12              
4  POINT (30.97520 -24.95397)         E             1.691478e+12              

   Distance_1  Distance_t Done Done_1  FID_  ...    Val_id Val_id_1  \
0           0           0                 0  ...  Val_4440            
1           0           0                 0  ...  Val_4441            
2           0           5    Y            0  ...    Val_19            
3           0          30    Y            0  ...    Val_23            
4           0          15    Y            0  ...    Val_33            

   Validation          X          Y        Z  Z_1  coords_x1 coords_x2  \
0           0   0.000000

In [11]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI',];

# Sample regions from the image at training feature locations
training_subset = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

Classification using random forest

In [12]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_subset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

Defining a colour palette for the classified map

In [13]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m




Map(center=[-24.91471009556816, 30.694320050468605], controls=(WidgetControl(options=['position', 'transparent…